<font size=8 face="courier" color="magenta">Setup Notebook</font>

This notebooks imports all the necessary packages to run cytoscape. It's also changes the display of the notebook.

# Import Packages

<font color=red>Check that this works</font>

In [3]:
from ipywidgets import *

Generic Python packages

In [4]:
import pandas as pd
import numpy as np
import time

import sys
import os

import inspect
import json
import uuid
import urllib.parse

`timeago` puts time in a "time since" format

In [1]:
import timeago

jupyter packages

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as ipw
import ipywidgets as widgets
from IPython.core.display import HTML, display, Javascript, clear_output

In [6]:
from IPython.utils.py3compat import str_to_bytes, bytes_to_str

<font color="magenta">Helper function</font> <font color="blue">showSource</font> shows source code of function

In [3]:
def showSource(func):
    print( inspect.getsource( func ) )

<font color="magenta">Helper function</font> <font color="green">to run</font> javavscript commands

In [7]:
def JS(text):
    display(HTML("<script>"+text+"</script>"))

<font color="magenta">Helper function</font> <font color="blue">to show</font> javavscript commands

In [8]:
def viewJS(text):
    display(Javascript("element.text(JSON.stringify("+text+"))"))

This line make is shorter to run the *kernel.execute* command, which is used to run python from javascript

In [9]:
JS("var py = Jupyter.notebook.kernel")

ndex packages for get data to/from ndex

In [10]:
import ndex2

github package for getting data onto github

In [11]:
from github import Github

# Setup Display
<font color="red">Note:</font> to reverse this feature you must rerun this cell with different numbers

make `display(HTML())` not take up any space when ran for styles and javascript

In [12]:
display(HTML("""<style>
div.output_subarea {
    padding: 0;
}
div.run_this_cell{
    padding: 0;
}
</style>"""))



remove the enormous margins on both sides of the notebook

In [13]:
display(HTML("""<style>
*{margin:0; padding:0;} html, body, \
             .container{margin:0;!important padding:0;!important} \
             .container { width:100% !important;}
</style>"""))

Change font size of text cells when rendered

In [14]:
display(HTML("""<style>
div.text_cell.rendered { /* Customize text cells */
    font-size:1.1em;
    line-height:1.4em;
}
</style>"""))

Make Container Background white

In [15]:
display(HTML("""<style>
#notebook-container {
    box-shadow: 0px 0px 0px 0px rgba(87, 87, 87, 0.2);
}
div#notebook {
    background-color: White;
}
.end_space {
    background-color: White;
}
</style>"""))

We make the little box to the left of the cell as small as possible

In [16]:
display(HTML("""<style>
.input_prompt {
    margin:0;
    padding:0;
    min-width: 0ex;
    font-size:.7em;
}
.prompt {
    min-width: 2vw;
}
div.input_prompt bdi {
    display: none;
}
div.input_prompt i {
    font-size: 2em;
    opacity: 0.7;
}

div.output_subarea { /* Makes html output width larger */
    max-width: 100%
}
</style>"""))

## Ipad Setup

In [17]:
display(Javascript("""
// Movement between Cells
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('-', 'jupyter-notebook:select-previous-cell');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('=', 'jupyter-notebook:select-next-cell');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('shift--', 'jupyter-notebook:extend-selection-above');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('shift-=', 'jupyter-notebook:extend-selection-below');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('alt--', 'jupyter-notebook:move-selection-up');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('alt-=', 'jupyter-notebook:move-selection-down');

// Controls Collapssing
Jupyter.keyboard_manager.command_shortcuts.add_shortcut(']', 'collapsible_headings:uncollapse_heading');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('[', 'collapsible_headings:collapse_heading');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('shift-]', 'collapsible_headings:uncollapse_all_headings');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('shift-[', 'collapsible_headings:collapse_all_headings');

// SKETCHY ONE // Entre command mode
Jupyter.keyboard_manager.edit_shortcuts.add_shortcut('shift-backspace', 'jupyter-notebook:enter-command-mode');
"""))

<IPython.core.display.Javascript object>

make scrolling WAY easier

In [18]:
display(HTML("""<style>
body {
    -webkit-overflow-scrolling: touch;
}
</style>"""))

make <font color="green">run</font> icons bigger

In [19]:
display(HTML("""<style>
div.run_this_cell i {
    font-size: 2em;
    opacity: 0.7;
}
</style>"""))

# Helpers

## Helper: <font color="magenta">JupyterCell</font>

<p></p>
 <font color="magenta">JupyterCell</font> is a useful class that allows you to execute a specified *jupyter cell* from the command <font color="magenta">JupyterCell</font><font color="blue">.execute()</font>. This is used during development to run cells later in a notebook (see [visualize code](/notebooks/Source%20Code/Visualize.ipynb)).

In [20]:
class JupyterCell():
 
    def __init__(self, cell_text, js_var, text_before="# ", ):
        self.cell_text= cell_text
        self.text_before= text_before
        self.js_var = js_var
        self.search()
    
    def search(self):
        JS("""
        var cell_text= "";
        """+self.js_var+"""= 0;  
        while( ! cell_text.includes('"""+self.text_before+self.cell_text+"""')  ){
            cell_text = Jupyter.notebook.get_cell("""+self.js_var+""").get_text();  //.element["0"].innerText #<-- Old Way
            """+self.js_var+"""++
        }
        """+self.js_var+"""--
        """)
    
    def execute(self):  #<-- This gets the Cell by using the cell text. The cell text must be unique to that cell
        self.search()
        JS("""IPython.notebook.execute_cells(["""+self.js_var+"""])""")

## Helper <font color="magenta">Render Json</font>

The class <font color="magenta">RenderJSON</font> creates an interactive display for JSON objects

In [21]:
class renderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

## Helper <font color="magenta">Run New Cell</font>

In [22]:
import base64
from IPython.utils.py3compat import str_to_bytes, bytes_to_str

In [23]:
def RunNewCell(code=''):
    encoded_code = bytes_to_str(base64.b64encode(str_to_bytes(code)))
    display(Javascript(f"""
        var code = Jupyter.notebook.insert_cell_below('code');
        code.set_text(atob("{encoded_code}"));
        code.metadata.cell_style = 'split';
        code.element.attr('style', 'float:left; width:50%;' );
        var cell_index = Jupyter.notebook.find_cell_index(code)        
        Jupyter.notebook.execute_cells([cell_index])
    """))

## Helper Embed <font color="darkKhaki">CY.HTML</font>

In [24]:
def embedFile(file_name, size="medium"):
    sizes = {'small':"col-md-offset-4 col-md-4", 'medium':"col-md-offset-3 col-md-6", 'large':"col-md-offset-2 col-md-8"}
    my_size = sizes[size]
    display(HTML(f"""
    <div class="row"><div class="col-xs-12 {my_size}">
    <div class="embed-responsive embed-responsive-4by3" style='border-style: solid;border-width: 1px;'>
      <iframe class="embed-responsive-item" 
      src='{file_name}'></iframe></div>
    </div></div>
    """))